Pulling data from OpenStreetMaps.org using osmxtract

In [1]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Is_Seattle_Walkable")

In [2]:
!pip install osmnx

  Using cached osmnx-1.1.2-py2.py3-none-any.whl (95 kB)


In [27]:
# import packages
import pandas as pd
import json
import osmnx as ox
import requests
import random
import time

In [28]:
leisure_gdf = ox.geometries_from_place('Seattle, Washington',{'leisure':True})
                                      

/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [29]:
leisure_gdf.leisure.unique()

array(['slipway', 'picnic_table', 'fitness_centre', 'playground',
       'fishing', 'marina', 'dance', 'sports_centre', 'amusement_arcade',
       'social_club', 'swimming_pool', 'escape_game', 'martial_arts',
       'art_gallery', 'games', 'table', 'dog_park', 'pitch', 'park',
       'fitness_station', 'hackerspace', 'bleachers', 'garden', 'firepit',
       'outdoor_seating', 'sports_hall', 'golf_course', 'stadium', 'farm',
       'nature_reserve', 'boating', 'splash_pool', 'ice_rink',
       'community_centre', 'bowling_alley', 'swimming_area', 'spraypark',
       'proposed', 'horse_riding', 'common', 'miniature_golf',
       'disc_golf_course', 'practice_tent', 'sauna', 'schoolyard',
       'track'], dtype=object)

In [30]:
leisure_gdf.shape

(3090, 193)

In [31]:
print(leisure_gdf.columns.tolist())

['source', 'geometry', 'created_by', 'leisure', 'seamark:information', 'seamark:small_craft_facility:category', 'seamark:type', 'direction', 'highway', 'barrier', 'access', 'bicycle', 'foot', 'name', 'addr:housenumber', 'addr:street', 'gnis:county_id', 'gnis:feature_id', 'gnis:created', 'gnis:state_id', 'ele', 'fee', 'opening_hours', 'website', 'artwork_type', 'description', 'location', 'tourism', 'sport', 'addr:city', 'addr:postcode', 'operator', 'fixme', 'horse', 'brand', 'brand:wikidata', 'brand:wikipedia', 'wheelchair', 'addr:state', 'seamark:harbour:category', 'addr:unit', 'covered', 'capacity', 'checked_exists:date', 'source:addr', 'addr:country', 'level', 'phone', 'amusement_arcade', 'contact:facebook', 'contact:instagram', 'contact:twitter', 'amenity', 'contact:website', 'dance:teaching', 'source:addr:id', 'disused:name', 'disused:shop', 'contact:email', 'contact:phone', 'contact:pinterest', 'contact:youtube', 'indoor', 'fax', 'shop', 'alt_name', 'office', 'wikidata', 'wikipedi

In [45]:
leisure_gdf = leisure_gdf.reset_index().loc[: , ['leisure','geometry','name']]

In [8]:
amenity_gdf = ox.geometries_from_place('Seattle, Washington',{'amenity': ['theatre', 
                                                                          'bar',
                                                                          'cafe',
                                                                          'pub',
                                                                          'restaurant',
                                                                          'school',
                                                                          'library',
                                                                          'cinema', 
                                                                          'fountain',
                                                                          'arts_centre',
                                                                          'community_centre',
                                                                          'events_venue']})

/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [9]:
amenity_gdf.shape

(3023, 267)

In [49]:
amenity_gdf = amenity_gdf.reset_index().loc[: , ['amenity','geometry','name']]

In [50]:
amenity_gdf.amenity.unique()

array(['cinema', 'theatre', 'restaurant', 'pub', 'school', 'cafe',
       'arts_centre', 'library', 'community_centre', 'bar', 'fountain',
       'events_venue'], dtype=object)

In [52]:
leisure_gdf.leisure.unique()

array(['slipway', 'picnic_table', 'fitness_centre', 'playground',
       'fishing', 'marina', 'dance', 'sports_centre', 'amusement_arcade',
       'social_club', 'swimming_pool', 'escape_game', 'martial_arts',
       'art_gallery', 'games', 'table', 'dog_park', 'pitch', 'park',
       'fitness_station', 'hackerspace', 'bleachers', 'garden', 'firepit',
       'outdoor_seating', 'sports_hall', 'golf_course', 'stadium', 'farm',
       'nature_reserve', 'boating', 'splash_pool', 'ice_rink',
       'community_centre', 'bowling_alley', 'swimming_area', 'spraypark',
       'proposed', 'horse_riding', 'common', 'miniature_golf',
       'disc_golf_course', 'practice_tent', 'sauna', 'schoolyard',
       'track'], dtype=object)

In [56]:
#cut down on redundant and unwalkable leisure activities ex. Boat Ramps
leisure_walk = ['picnic_table', 'fitness_centre','fishing', 'marina', 'dance', 
                'sports_centre', 'amusement_arcade','social_club', 'swimming_pool', 'escape_game',
                'martial_arts', 'art_gallery', 'games', 'table', 'fitness_station', 
                'hackerspace', 'garden', 'firepit', 'outdoor_seating', 'sports_hall',
                'farm','nature_reserve', 'boating','ice_rink', 'community_centre', 
                'bowling_alley', 'swimming_area', 'spraypark','horse_riding', 'miniature_golf', 
                'practice_tent', 'sauna', 'track']
leisure_ldf = leisure_gdf.loc[leisure_gdf['leisure'].isin(leisure_walk)]


,leisure,geometry,name
4,picnic_table,POINT (-122.33925 47.69838),NaN
5,picnic_table,POINT (-122.33917 47.69844),NaN
8,picnic_table,POINT (-122.33788 47.69938),NaN
9,picnic_table,POINT (-122.33807 47.69860),NaN
10,picnic_table,POINT (-122.33821 47.69803),NaN
...,...,...,...
3078,fitness_station,"MULTIPOLYGON (((-122.36840 47.54603, -122.3683...",NaN
3079,track,"POLYGON ((-122.33832 47.69586, -122.33833 47.6...",NaN
3083,track,"POLYGON ((-122.29838 47.69120, -122.29770 47.6...",NaN
3084,track,"POLYGON ((-122.29494 47.68116, -122.29494 47.6...",NaN


In [59]:
leisure_gdf = leisure_gdf.rename(columns={'leisure': 'amenity'})

In [64]:
cult_ent_gdf = pd.concat([leisure_gdf,amenity_gdf])

In [81]:
cult_ent_gdf.to_file('culture_leisure_amenities',driver='GeoJSON')

## Pulling Retail Data

In [66]:
retail_gdf = ox.geometries_from_place('Seattle, Washington',{'shop':True})

/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [68]:
retail_gdf.shape

(3524, 218)

In [71]:
retail_gdf = retail_gdf.reset_index().loc[: , ['shop','geometry','name']]

In [72]:
retail_gdf.head()

,shop,geometry,name
0,books,POINT (-122.31298 47.66040),University Book Store
1,car_repair,POINT (-122.30147 47.70906),Midas
2,supermarket,POINT (-122.29521 47.72164),Grocery Outlet
3,car,POINT (-122.30051 47.71129),Pierre Chevrolet
4,bakery,POINT (-122.38656 47.56122),Cupcake Royale


In [80]:
retail_gdf.to_file('seattle_retail',driver='GeoJSON')